In [1]:
!pip install tsfresh
!pip install catboost

    100% |████████████████████████████████| 1.2MB 9.1MB/s 
    100% |████████████████████████████████| 55.5MB 793kB/s 


In [2]:
# The essentials
import pandas as pd
import numpy as np

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Progress bars
from tqdm import tqdm

# Access our Google Drive
from google.colab import drive

# Gradient Boosting
from catboost import CatBoostRegressor

from collections import defaultdict

from tsfresh.feature_selection.relevance import calculate_relevance_table

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/Rinse Over Run"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
20178.png
20451.png
20899.png
22112.png
22369.png
22414.png
22487.png
23011.png
23142.png
23599.png
23872.png
24804.png
24845.png
24872.png
25129.png
25908.png
25983.png
26270.png
27115.png
27243.png
27346.png
27366.png
27418.png
27508.png
all_train_preds_per_phase.p
baseline_features_with_preds_per_phase.csv
baseline_model_per_nunique_phases.csv
dtw_distances_3.p
extended_phase_predictors.csv
last_cleaned_test.csv
last_cleaned_train.csv
mds_embeddings_2d_3.cs

In [4]:
train_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_values.csv', index_col=0, parse_dates=['timestamp'])
test_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/test_values.csv', index_col=0, parse_dates=['timestamp'])
label_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_labels.csv', index_col='process_id')

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
process_comb = 3
train_features_index = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_features_adv_{}.csv'.format(process_comb), index_col=['process_id']).index
val_features_index = pd.read_csv('/content/drive/My Drive/Rinse Over Run/val_features_adv_{}.csv'.format(process_comb), index_col=['process_id']).index
test_features_index = pd.read_csv('/content/drive/My Drive/Rinse Over Run/test_features_{}.csv'.format(process_comb), index_col=['process_id']).index

In [0]:
ts_real = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'target_value'
]

# variables we'll use to create our time series features
ts_cols = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'target_value'
]

# variables for binary time series features
bin_cols = [
    'supply_pump',
    'supply_pre_rinse',
    'supply_caustic',
    'return_caustic',
    'supply_acid',
    'return_acid',
    'supply_clean_water',
    'return_recovery_water',
    'return_drain',
    'object_low_level'
]

process_comb_to_phases = {
    15: ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid'],
    3:  ['pre_rinse', 'caustic'],
    7:  ['pre_rinse', 'caustic', 'intermediate_rinse'],
    1:  ['pre_rinse'],
    8:  ['acid'],
    2:  ['caustic'],
    6:  ['caustic', 'intermediate_rinse'],
    14: ['caustic', 'intermediate_rinse', 'acid'],
}

# phases, ordered from earliest to latest
phases = ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid']

def encode_categorical(df):
    # Currently just copy-pasted from http://drivendata.co/blog/rinse-over-run-benchmark/
    
    # select process_id and pipeline
    meta = df[['process_id', 'pipeline']].drop_duplicates().set_index('process_id') 
    
    # convert categorical pipeline data to dummy variables
    meta = pd.get_dummies(meta)
    
    # pipeline L12 not in test data (so useless feature)
    if 'pipeline_L12' in meta:
        meta = meta.drop('pipeline_L12', axis=1)
    
    # calculate number of phases for each process_object
    meta['num_phases'] = df.groupby('process_id')['phase'].apply(lambda x: x.nunique())
    
    return meta

def percentile_25(x):
  return np.percentile(x, 0.25)

def percentile_75(x):
  return np.percentile(x, 0.75)
  
def encode_real_timeseries(df):   
    ts_df = df[['process_id'] + ts_cols].set_index('process_id')
    
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', 
                                                   'count', 'median', 'sum', 
                                                   'mad'])
    
    # Now we will get the mean + variance value of the last K measurements for each phase
    # TODO: This can be moved to extracting features per phase
    all_vals_per_phase = []
    K = 5
    col_names = ['process_id'] 
    for phase in phases:
        for col in ts_cols:
            col_names.extend(['mean_{}_{}_{}'.format(col, K, phase), 
                              'std_{}_{}_{}'.format(col, K, phase)])
    for process in tqdm(ts_features.index, total=len(ts_features)):
        vals_per_phase = [process]
        process_filtered_df = df[df['process_id'] == process]
        for phase in phases:
            filtered_df = process_filtered_df[process_filtered_df['phase'] == phase].tail(K)
            for col in ts_cols:
                vals_per_phase.extend([filtered_df[col].mean(), filtered_df[col].std()])
                
        all_vals_per_phase.append(vals_per_phase)
    values_df = pd.DataFrame(all_vals_per_phase, columns=col_names)
    values_df = values_df.set_index('process_id')
    
    ts_features = ts_features.merge(values_df, left_index=True, right_index=True)
    
    col_map = {}
    for col in ts_features.columns:
        col_map[col] = 'real_{}'.format(col)
    ts_features = ts_features.rename(columns=col_map)
    
    return ts_features

def encode_binary_timeseries(df):
    ts_df = df[['process_id'] + bin_cols].set_index('process_id')
            
    # create features: count, min, max, mean, standard deviation
    ts_features = ts_df.groupby('process_id').agg(['mean', 'std', 'count', 'sum', 'mad'])
    
    # TODO: Count fraction of True in each phase
    feature_vectors = []
    col_names = ['process_id'] 
    for phase in phases:
        for col in bin_cols:
            col_names.append('fraction_{}_{}'.format(col, phase))
            
    # Get fraction of True values for each binary timeseries
    # TODO: This can be moved to extracting features per phase
    for process in tqdm(set(df['process_id']), total=len(set(df['process_id']))):
        vector = [process]
        process_filtered_df = df[df['process_id'] == process]
        for phase in phases:
            filtered_df = process_filtered_df[process_filtered_df['phase'] == phase]
            for col in bin_cols:
                if len(filtered_df):
                    vector.append(sum(filtered_df[col]) / len(filtered_df))
                else:
                    vector.append(np.NaN)
                
        feature_vectors.append(vector)
                
    feature_df = pd.DataFrame(feature_vectors, columns=col_names)
    feature_df = feature_df.set_index('process_id')
    
    feature_df = feature_df.merge(ts_features, left_index=True, right_index=True)
    col_map = {}
    for col in feature_df.columns:
        col_map[col] = 'bin_{}'.format(col)
    feature_df = feature_df.rename(columns=col_map)
    
    return feature_df
  

def get_descript(data, functions, cols):
    ts_df = data.set_index('process_id').sort_values(by='timestamp')
    return ts_df.groupby('process_id')[cols].agg(functions)  
  
  
def get_descript_prev_process(data):
    machines = set(data['object_id'])
    all_features = []
    for machine in tqdm(machines):
        machine_data = data[data['object_id'] == machine]
        machine_data = machine_data.sort_values(by='timestamp')
        machine_processes = machine_data['process_id'].unique()
        for process_ix, process in enumerate(machine_processes):
            if process_ix > 0:
                prev_process = machine_data[machine_data['process_id'] == machine_processes[process_ix - 1]]
                this_process = machine_data[machine_data['process_id'] == machine_processes[process_ix]]
                features = get_descript(prev_process, ['mean', 'std', 'min', 'max', 'count'], ts_real)
                _columns = list(features.columns)
                assert len(features) == 1
                features = features.iloc[0, :].values
                time_delta = (this_process['timestamp'].values[0] - prev_process['timestamp'].values[-1]) / np.timedelta64(1, 'h')
                assert time_delta > 0
                all_features.append([machine, process, time_delta] + list(features))
            else:
                all_features.append([machine, process, np.NaN] + ([np.NaN] * 60))
                
    all_features = pd.DataFrame(all_features, columns=['object_id', 'process_id', 'time_delta'] + _columns)
    all_features = all_features.set_index('process_id', drop=True)
    col_map = {}
    for col in all_features.columns:
        col_map[col] = 'prev_{}'.format(col)
    all_features = all_features.rename(columns=col_map)
    return all_features
  
def get_descript_per_phase(data, phases):
    all_features = pd.DataFrame(index=list(set(data['process_id'])))
    for phase in phases:
        phase_data = data[data['phase'] == phase].sort_values(by='timestamp')[['process_id'] + ts_real + bin_cols]
        col_map = {}
        for col in phase_data.columns:
          if col != 'process_id':
            col_map[col] = '{}_{}'.format(phase, col)
        phase_data = phase_data.rename(columns=col_map)
        features = phase_data.groupby('process_id').agg(['mean', 'std', 'count'])
        all_features = all_features.merge(features, left_index=True, right_index=True, how='left')
        
    col_map = {}
    for col in all_features.columns:
        col_map[col] = 'phase_{}'.format(col)
    all_features = all_features.rename(columns=col_map)

    return all_features

def create_feature_matrix(df):
    df['return_flow_relu'] = df['return_flow'].apply(lambda x: max(0, x))
    df['target_value'] = df['return_flow_relu'] * df['return_turbidity']
    
    #phase_features = get_descript_per_phase(df, ['pre_rinse', 'caustic'])
    prev_features = get_descript_prev_process(df)
    metadata = encode_categorical(df)
    time_series = encode_real_timeseries(df)
    binary_features = encode_binary_timeseries(df)
    
    # join metadata and time series features into a single dataframe
    feature_matrix = metadata
    feature_matrix = feature_matrix.merge(time_series, left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(binary_features, left_index=True, right_index=True)
    feature_matrix = feature_matrix.merge(prev_features, left_index=True, right_index=True)
    #feature_matrix = feature_matrix.merge(phase_features, left_index=True, right_index=True)
    
    return feature_matrix

In [48]:
all_train_index = list(train_features_index) + list(val_features_index)
train_df_filtered = train_df[(train_df['phase'].isin(process_comb_to_phases[process_comb])) &
                             (train_df['process_id'].isin(all_train_index))]
test_df_filtered = test_df[(test_df['phase'].isin(process_comb_to_phases[process_comb])) &
                           (test_df['process_id'].isin(test_features_index))]

features = create_feature_matrix(pd.concat([train_df_filtered, test_df_filtered]))

X_train = features.loc[train_features_index]
X_val = features.loc[val_features_index]
X_test = features.loc[test_features_index]


np.random.seed(1337)
X_train = pd.concat([X_train, X_val])

val_idx = np.random.choice(list(X_train.index), replace=False, size=int(0.1*len(X_train)))
train_idx = list(set(X_train.index) - set(val_idx))

X_val = X_train.loc[val_idx]
X_train = X_train.loc[train_idx]

100%|██████████| 5925/5925 [02:04<00:00, 47.48it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:558: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)
100%|██████████| 5925/5925 [00:55<00:00, 106.25it/s]
/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [0]:
y_train = np.log(label_df.loc[X_train.index])
y_val = np.log(label_df.loc[X_val.index])


y_train = pd.concat([y_train, y_val], axis=0)
y_val = y_train.loc[val_idx]
y_train = y_train.loc[train_idx]

In [55]:
# Let's get the highly correlated features first
def get_corr_features(X):
  row_idx, col_idx = np.where(X.corr() == 1)
  self_corr = set([(i, i) for i in range(X_train.shape[1])])
  return set(list(zip(row_idx, col_idx))) - self_corr 

X_train_uncorr = X_train.copy()
correlated_features = get_corr_features(X_train_uncorr)
while correlated_features:
  print('{} correlated feature pairs left...'.format(len(correlated_features)))
  corr_row, corr_col = correlated_features.pop()
  print('{} is correlated with {}... Removing {}.'.format(X_train_uncorr.columns[corr_row], X_train_uncorr.columns[corr_col], X_train_uncorr.columns[corr_row]))
  col_mask = [True]*X_train_uncorr.shape[1]
  col_mask[corr_row] = False
  X_train_uncorr = X_train_uncorr.loc[:, col_mask]
  correlated_features = get_corr_features(X_train_uncorr)
  
print(X_train.shape, X_train_uncorr.shape)

890 correlated feature pairs left...
bin_('object_low_level', 'count') is correlated with real_('tank_level_caustic', 'count')... Removing bin_('object_low_level', 'count').
840 correlated feature pairs left...
prev_('tank_temperature_pre_rinse', 'count') is correlated with prev_('tank_level_acid', 'count')... Removing prev_('tank_temperature_pre_rinse', 'count').
810 correlated feature pairs left...
real_('tank_temperature_caustic', 'count') is correlated with real_('supply_flow', 'count')... Removing real_('tank_temperature_caustic', 'count').
762 correlated feature pairs left...
real_('tank_level_caustic', 'count') is correlated with bin_('return_caustic', 'count')... Removing real_('tank_level_caustic', 'count').
716 correlated feature pairs left...
real_('return_turbidity', 'count') is correlated with bin_('return_acid', 'count')... Removing real_('return_turbidity', 'count').
672 correlated feature pairs left...
bin_('supply_pre_rinse', 'count') is correlated with bin_('return_ac

In [60]:
# Now let's remove columns with too many NA's and only 1 single value
#na_cols = X_train_uncorr.columns[X_train_uncorr.isnull().sum() / len(X_train_uncorr) > 0.5]
single_cols = X_train_uncorr.columns[X_train_uncorr.nunique() == 1]

uncorr_cols = X_train_uncorr.columns
uncorr_cols = list(set(uncorr_cols) - set(single_cols)) #.union(set(na_cols))

#print(uncorr_cols)

X_train_no_corr = X_train[uncorr_cols]
X_val_no_corr = X_val[uncorr_cols]
print(X_train.shape, X_train_no_corr.shape)
#X_train.to_csv('/content/drive/My Drive/Rinse Over Run/train_features_sel_{}.csv'.format(process_comb))
#X_val.to_csv('/content/drive/My Drive/Rinse Over Run/val_features_sel_{}.csv'.format(process_comb))

(4269, 439) (4269, 383)


In [0]:
rel_table = calculate_relevance_table(X_train.dropna(axis=1), y_train['final_rinse_total_turbidity_liter'], ml_task='regression')

In [0]:
X_train_na_cols = X_train.columns[X_train.isnull().sum() > 1]
X_train_no_na = X_train[X_train_na_cols].dropna(axis=0)
rel_table_na = calculate_relevance_table(X_train_no_na.astype(float), y_train.loc[X_train_no_na.index]['final_rinse_total_turbidity_liter'], ml_task='regression')

In [43]:
X_train_selected_cols = list(set(rel_table_na[rel_table_na['p_value'] < 0.05]['feature']).union(set(rel_table[rel_table['p_value'] < 0.05]['feature'])))
len(X_train_selected_cols), len(X_train.columns)

(293, 378)

In [44]:
print(X_train_selected_cols)

X_train_sub = X_train[X_train_selected_cols]
X_val_sub = X_val[X_train_selected_cols]
X_test_sub = X_test[X_train_selected_cols]
#X_train.to_csv('/content/drive/My Drive/Rinse Over Run/train_features_sel_{}.csv'.format(process_comb))
#X_val.to_csv('/content/drive/My Drive/Rinse Over Run/val_features_sel_{}.csv'.format(process_comb))
#X_test.to_csv('/content/drive/My Drive/Rinse Over Run/test_features_sel_{}.csv'.format(process_comb))

["prev_('tank_level_caustic', 'min')", "real_('tank_concentration_caustic', 'sum')", "real_('return_flow', 'sum')", "phase_('caustic_return_flow', 'std')", "real_('tank_level_clean_water', 'mad')", 'real_mean_supply_pressure_5_pre_rinse', "prev_('supply_flow', 'max')", 'real_std_tank_level_pre_rinse_5_pre_rinse', "prev_('return_conductivity', 'min')", "real_('tank_concentration_acid', 'sum')", "real_('return_turbidity', 'mad')", "real_('tank_concentration_acid', 'mad')", "prev_('tank_concentration_caustic', 'mean')", "prev_('tank_level_caustic', 'mean')", "real_('target_value', 'sum')", "prev_('tank_temperature_acid', 'min')", "real_('tank_concentration_acid', 'std')", "prev_('supply_flow', 'mean')", 'real_std_tank_concentration_caustic_5_pre_rinse', 'pipeline_L11', "real_('tank_concentration_caustic', 'std')", "real_('target_value', 'mad')", "bin_('supply_caustic', 'std')", "real_('tank_level_clean_water', 'median')", "real_('supply_pressure', 'std')", "bin_('object_low_level', 'mad')

In [0]:
def custom_mape(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

class MAPEMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, targets, weight):
        return custom_mape(np.exp(approxes), np.exp(targets)), len(targets)

In [61]:
cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=250, learning_rate=0.33,
                        loss_function='MAPE', eval_metric='MAPE', border_count=254, task_type='GPU')#, l2_leaf_reg=10) #MAPEMetric()
cat.fit(X_train_no_corr, y_train, eval_set=(X_val_no_corr, y_val), verbose=50)

# baseline: 0.3098982664741879
# removing useless features: 0.31096976977748236
# more features: 0.3098482619660084

0:	learn: 0.9981617	test: 0.9981692	best: 0.9981692 (0)	total: 73.4ms	remaining: 2h 2m 18s
50:	learn: 0.9055828	test: 0.9059201	best: 0.9059201 (50)	total: 2.4s	remaining: 1h 18m 34s
100:	learn: 0.8127572	test: 0.8136417	best: 0.8136417 (100)	total: 4.04s	remaining: 1h 6m 34s
150:	learn: 0.7199331	test: 0.7214031	best: 0.7214031 (150)	total: 5.61s	remaining: 1h 1m 47s
200:	learn: 0.6271076	test: 0.6290847	best: 0.6290847 (200)	total: 7.3s	remaining: 1h 23s
250:	learn: 0.5342897	test: 0.5368148	best: 0.5368148 (250)	total: 8.95s	remaining: 59m 18s
300:	learn: 0.4415060	test: 0.4445416	best: 0.4445416 (300)	total: 10.6s	remaining: 58m 40s
350:	learn: 0.3506329	test: 0.3535772	best: 0.3535772 (350)	total: 12.5s	remaining: 59m 19s
400:	learn: 0.2664131	test: 0.2677799	best: 0.2677799 (400)	total: 15.4s	remaining: 1h 3m 51s
450:	learn: 0.1927465	test: 0.1932633	best: 0.1932633 (450)	total: 18.4s	remaining: 1h 7m 47s
500:	learn: 0.1251733	test: 0.1246239	best: 0.1246239 (500)	total: 21.5s	re

In [62]:
print(custom_mape(np.exp(cat.predict(X_val)), np.exp(y_val.values.flatten())))

0.3113927192320934


In [36]:
single_cols

Index(['num_phases', 'bin_fraction_return_caustic_pre_rinse',
       'bin_fraction_supply_acid_pre_rinse',
       'bin_fraction_return_acid_pre_rinse',
       'bin_fraction_supply_clean_water_pre_rinse',
       'bin_fraction_return_recovery_water_pre_rinse',
       'bin_fraction_supply_acid_caustic',
       'bin_fraction_return_recovery_water_caustic',
       'bin_('supply_acid', 'mean')', 'bin_('supply_acid', 'std')',
       'bin_('supply_acid', 'sum')', 'bin_('supply_acid', 'mad')',
       'bin_('return_recovery_water', 'mean')',
       'bin_('return_recovery_water', 'std')',
       'bin_('return_recovery_water', 'sum')',
       'bin_('return_recovery_water', 'mad')',
       'phase_('pre_rinse_return_caustic', 'mean')',
       'phase_('pre_rinse_return_caustic', 'std')',
       'phase_('pre_rinse_supply_acid', 'mean')',
       'phase_('pre_rinse_supply_acid', 'std')',
       'phase_('pre_rinse_return_acid', 'mean')',
       'phase_('pre_rinse_return_acid', 'std')',
       'phase_('pr

In [50]:
# cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=250, learning_rate=0.33,
#                         loss_function='MAPE', eval_metric='MAPE')#
# cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=50)

0:	learn: 0.9981518	test: 0.9981495	best: 0.9981495 (0)	total: 29.9ms	remaining: 49m 54s


KeyboardInterrupt: ignored

In [0]:
rel_table_na

,feature,type,p_value,relevant
feature,,,,
"prev_('supply_flow', 'mean')","prev_('supply_flow', 'mean')",real,1.734781e-294,True
"prev_('return_flow', 'mean')","prev_('return_flow', 'mean')",real,5.860941e-282,True
"prev_('supply_flow', 'max')","prev_('supply_flow', 'max')",real,1.313609e-274,True
"prev_('supply_flow', 'std')","prev_('supply_flow', 'std')",real,8.289771e-224,True
"prev_('return_flow', 'std')","prev_('return_flow', 'std')",real,1.155660e-137,True
"prev_('return_flow', 'max')","prev_('return_flow', 'max')",real,4.138401e-101,True
"prev_('target_value', 'mean')","prev_('target_value', 'mean')",real,3.557386e-84,True
"prev_('supply_pressure', 'min')","prev_('supply_pressure', 'min')",real,1.161656e-59,True
"prev_('return_flow', 'min')","prev_('return_flow', 'min')",real,6.827346e-55,True
